In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                             width_shift_range=0.1,
                             height_shift_range=0.1, 
                             rotation_range=360,
                             horizontal_flip=True,
                             vertical_flip=True)

train_datagen.augmentation_parameters = True

validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

# Define the path to the dataset
train_path = './train'
validation_path = './validation'
test_path = './test'

# Prepare data generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(299, 299),
    batch_size=16,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(299, 299),
    batch_size=16,
    class_mode='categorical')

test_generator = validation_datagen.flow_from_directory(
    test_path,
    target_size=(299, 299),
    batch_size=16,
    class_mode='categorical')

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(299, 299, 3)), #this layer has 16 filters with size of (3,3)
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'), #The ReLU activation function replaces all negative values in the input with zero, and outputs the input value if it is positive.
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), # Turns layers into one dimensional array
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

In [ ]:
# Train model using generators
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=60,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    )

In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print('Test Loss: ', test_loss)
print('Test Accuracy: ', test_acc)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="darkgrid")

# Create a figure and axis
fig, ax = plt.subplots(dpi=200)
fig.set_size_inches(1920/fig.dpi, 1080/fig.dpi)

# Training accuracy
ax.stackplot(range(1, len(history.history['accuracy']) + 1), 
             history.history['accuracy'], 
             labels=['Accuracy'],
             alpha=0.9, 
             colors=['#F35C16'])

# Validation accuracy
ax.stackplot(range(1, len(history.history['val_accuracy']) + 1), 
             history.history['val_accuracy'], 
             labels=['Validation Accuracy'],
             alpha=0.8, 
             colors=['#07004D'])

# Add title, labels and legend
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.ylim(0,1)
plt.legend(loc='upper left')

# Save the plot as an image with the name "accuracy_plot.png"
plt.savefig('CNN_model_01.png')
plt.show()

In [ ]:
# Save the model
#model.save('model01.h5')